In [3]:
!wget https://raw.githubusercontent.com/sonishko/hse22_hw2/main/data/gms2.lst

--2022-10-19 19:16:10--  https://raw.githubusercontent.com/sonishko/hse22_hw2/main/data/gms2.lst
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 219943 (215K) [text/plain]
Saving to: ‘gms2.lst’

gms2.lst            100%[===================>] 214.79K  --.-KB/s    in 0.009s  

2022-10-19 19:16:11 (23.9 MB/s) - ‘gms2.lst’ saved [219943/219943]



In [4]:
!wget https://raw.githubusercontent.com/sonishko/hse22_hw2/main/data/scaffolds.fa

--2022-10-19 19:16:13--  https://raw.githubusercontent.com/sonishko/hse22_hw2/main/data/scaffolds.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3955483 (3.8M) [text/plain]
Saving to: ‘scaffolds.fa’

scaffolds.fa        100%[===================>]   3.77M  --.-KB/s    in 0.03s   

2022-10-19 19:16:13 (126 MB/s) - ‘scaffolds.fa’ saved [3955483/3955483]



In [5]:
!wget https://raw.githubusercontent.com/sonishko/hse22_hw2/main/data/proteins.fasta

--2022-10-19 19:16:15--  https://raw.githubusercontent.com/sonishko/hse22_hw2/main/data/proteins.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1401032 (1.3M) [text/plain]
Saving to: ‘proteins.fasta’

proteins.fasta      100%[===================>]   1.34M  --.-KB/s    in 0.06s   

2022-10-19 19:16:15 (22.0 MB/s) - ‘proteins.fasta’ saved [1401032/1401032]



In [6]:
!wget https://raw.githubusercontent.com/sonishko/hse22_hw2/main/data/scaffolds.hits_from_MIL_1.txt

--2022-10-19 19:16:17--  https://raw.githubusercontent.com/sonishko/hse22_hw2/main/data/scaffolds.hits_from_MIL_1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 867036 (847K) [text/plain]
Saving to: ‘scaffolds.hits_from_MIL_1.txt’

scaffolds.hits_from 100%[===================>] 846.71K  --.-KB/s    in 0.02s   

2022-10-19 19:16:17 (52.8 MB/s) - ‘scaffolds.hits_from_MIL_1.txt’ saved [867036/867036]



In [7]:
!wget https://raw.githubusercontent.com/sonishko/hse22_hw2/main/data/scaffolds.hits_from_SwissProt.txt

--2022-10-19 19:16:19--  https://raw.githubusercontent.com/sonishko/hse22_hw2/main/data/scaffolds.hits_from_SwissProt.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65851 (64K) [text/plain]
Saving to: ‘scaffolds.hits_from_SwissProt.txt’

scaffolds.hits_from 100%[===================>]  64.31K  --.-KB/s    in 0.004s  

2022-10-19 19:16:19 (17.8 MB/s) - ‘scaffolds.hits_from_SwissProt.txt’ saved [65851/65851]



## Скачиваем геном близкородственной бактерии T.oleivorans
Геном, последовательности генов (нт) и белков (протеом) для бактерии Thalassolituus oleivorans MIL-1
https://www.ncbi.nlm.nih.gov/nuccore/HF680312

In [8]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
y
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [9]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [10]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Аннотация генома 

In [11]:
from Bio import SeqIO, SearchIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pandas as pd

seqs = list(SeqIO.parse("scaffolds.fa", "fasta"))
prots = list(SeqIO.parse("proteins.fasta", "fasta"))


Найдем кол-во скаффолдов

In [12]:
len(seqs)


68

Найдем длину максимального скаффолда

In [13]:
max_len = 0 
for seq in seqs:
  if len(seq) > max_len:
    max_len = len(seq)
max_len

3867647

Найдем общее кол-во генов

In [14]:
len(prots)

3602

Считываем скаффолды

In [15]:
scaffolds = {}
for record in SeqIO.parse('scaffolds.fa', 'fasta'):
  record.annotations['molecule_type'] = 'DNA'
  record.annotations['organism'] = 'Thalassolituus oleivorans Barents'
  record.annotations['data_file_division'] = 'BCT'
  scaffolds[record.id] = record

In [16]:
genes = dict()
for gene in SeqIO.parse("proteins.fasta", "fasta"):
  descr = gene.description.split(' ')
  scaffold = descr[1]
  start, end = int(descr[2]), int(descr[3])
  
  feature = SeqFeature(FeatureLocation(start, end), type="CDS")
  feature.qualifiers['locus_tag'] = [descr[0]]
  feature.qualifiers['translation'] = [gene.seq]
  scaffolds[scaffold].features.append(feature)
  
  genes[descr[0]] = feature

Добавляем функции белков (аннотацию) из MIL-1

In [17]:
mil1 = dict()
for item in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features:
  if 'protein_id' not in item.qualifiers:
    continue
  if 'product' not in item.qualifiers:
    continue
  
  mil1[item.qualifiers['protein_id'][0]] = item.qualifiers['product'][0]

Читаем scaffolds.hits_from_MIL1.txt

In [18]:
names = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
mil1_hits = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=names)


Выбираем лучшие совпадения для каждого гена

In [19]:
hit = mil1_hits[mil1_hits['sseqid'].str.contains("CCU")].sort_values('bitscore', ascending=False).drop_duplicates('qseqid')

for i, hit in hit.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('_')[2]
  gene.qualifiers['product'] = [mil1[match]]

## Добавляем функции белков (из БД SwissProt)

In [20]:
!head scaffolds.hits_from_SwissProt.txt

103	sp|Q58577|Y1178_METJA	30.055	366	192	15	1	336	4	335	2.97e-28	115
103	sp|Q81ST7|BSHA_BACAN	29.097	299	168	10	63	335	77	357	2.74e-22	100
103	sp|P9WMZ3|PIMB_MYCTU	28.689	244	148	6	113	336	130	367	3.03e-21	97.1
103	sp|P9WMZ2|PIMB_MYCTO	28.689	244	148	6	113	336	130	367	3.03e-21	97.1
103	sp|O32272|TUAC_BACSU	25.313	399	230	11	1	348	1	382	8.09e-21	95.9
103	sp|Q59002|Y1607_METJA	28.523	298	181	10	67	340	84	373	1.72e-18	89.4
103	sp|Q8NTA6|MSHA_CORGL	30.147	272	160	13	103	352	144	407	1.34e-17	86.7
103	sp|A4QB40|MSHA_CORGB	30.147	272	160	13	103	352	144	407	1.34e-17	86.7
103	sp|A7TZT2|MFPS_AGRFC	26.400	250	160	6	106	336	177	421	9.87e-17	84.3
103	sp|Q8FSH1|MSHA_COREF	29.588	267	156	12	106	349	147	404	3.79e-16	82.4


In [21]:
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

--2022-10-19 19:17:53--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642093634 (612M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 612.35M  19.6MB/s    in 27s     

2022-10-19 19:18:21 (22.7 MB/s) - ‘uniprot_sprot.dat.gz’ saved [642093634/642093634]



In [22]:
!gzip -d uniprot_sprot.dat.gz

In [23]:
!grep '^ID\|^DE   RecName: Full=' uniprot_sprot.dat > SwissProt_names.txt

In [24]:
swprgenes = dict()
for row in open('SwissProt_names.txt'):
  if row.startswith('ID'):
    previd = row.split()[1]
  if row.startswith('DE'):
    swprgenes[previd] = row.split('=')[1][:-2]

In [25]:
swpr_hits_raw = pd.read_csv('scaffolds.hits_from_SwissProt.txt', sep='\t', header=None, names=names)
swpr_hits = swpr_hits_raw.sort_values('bitscore', ascending=False).drop_duplicates('qseqid')

for i, hit in swpr_hits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('|')[-1]
  gene.qualifiers['product'] = [swprgenes[match]]

In [26]:
SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

68

Посмотрим на статистику

In [ ]:
len(list(SearchIO.parse("scaffolds.hits_from_MIL_1.txt", "blast-tab")))


3324

In [ ]:
len(list(SearchIO.parse("scaffolds.hits_from_SwissProt.txt", "blast-tab")))

53

In [ ]:
without_annotation = len(prots) - len(list(SearchIO.parse("scaffolds.hits_from_MIL_1.txt", "blast-tab"))) - len(list(SearchIO.parse("scaffolds.hits_from_SwissProt.txt", "blast-tab")))
without_annotation

225